In [3]:
import pandas as pd
from math import isnan

In [13]:
#get historical figures
historical_figures_train = pd.read_csv('historical_figures/train1.csv')
historical_figures_test = pd.read_csv('historical_figures/test.csv')
historical_figures = pd.concat([historical_figures_train, historical_figures_test], axis=0)

In [14]:
historical_figures.head()

,id,id.1,full_name,sex,birth_year,city,state,country,continent,latitude,longitude,occupation,industry,domain,article_languages,page_views,average_views,historical_popularity_index
0,4110,3728,Gao Xingjian,Male,1940,Ganzhou,NaN,China,Asia,25.86667,114.93333,Writer,Language,Humanities,63,1439340,22847,24.1041
1,8160,3769,Wassily Leontief,Male,1905,Munich,NaN,Germany,Europe,48.13333,11.56667,Economist,Social Sciences,Science & Technology,43,621949,14464,24.0845
2,4927,1777,Justus von Liebig,Male,1803,Darmstadt,NaN,Germany,Europe,49.86667,8.65000,Chemist,Natural Sciences,Science & Technology,44,1277543,29035,25.3021
3,3031,5856,William Stanley Jevons,Male,1835,Liverpool,NaN,United Kingdom,Europe,53.40000,-3.00000,Economist,Social Sciences,Science & Technology,27,556636,20616,22.9292
4,7267,58,Thomas Aquinas,Male,1225,Roccasecca,NaN,Italy,Europe,41.55000,13.66667,Philosopher,Philosophy,Humanities,103,17427900,169203,29.4352


In [15]:
historical_figures.columns

Index(['id', 'id.1', 'full_name', 'sex', 'birth_year', 'city', 'state',
       'country', 'continent', 'latitude', 'longitude', 'occupation',
       'industry', 'domain', 'article_languages', 'page_views',
       'average_views', 'historical_popularity_index'],
      dtype='object')

In [105]:
historical_figures = historical_figures[historical_figures['birth_year'] != 'Unknown']
historical_figures['birth_year'] = historical_figures['birth_year'].apply(lambda i: str(i).replace('?', ''))

In [4]:
# get fake bios
fake_data_path = '../epfl_server/fakes.csv'
fakes_df = pd.read_csv(fake_data_path, sep='|')
fakes_df.columns = ['bio']
fakes_df.head()

,bio
0,Constanzo Rossi was a Venetian Engineer. [SEP]...
1,Umberto Rizzo was a Venetian Forester. [SEP] ...
2,Adriano Rossi was a Venetian Moneylender. [SEP...
3,Adamo Bianchi was a Venetian Pardoner. [SEP] ...
4,Massimo Giordano was a Venetian Artist. [SEP] ...


In [18]:
example = fakes_df['bio'][0]

In [21]:
nlp = spacy.load('en')
# neuralcoref.add_to_pipe(nlp)
example_doc = nlp(example)
people = []
for ent in example_doc.ents:
    if ent.label_ == 'PERSON':
        people.append(ent)
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

Constanzo Rossi 0 15 PERSON
Giacomo Rossi 84 97 PERSON
Laura Scrini 102 114 PERSON
Sonntag 457 464 PERSON
Marino 898 904 PERSON
Arnold Hayez 1325 1337 PERSON
Friedrich Wilhelm Lechner 1342 1367 PERSON
Aldo Manuzio 1602 1614 PERSON
Claudio Monteverdi 1616 1634 PERSON
Francesco Hayez 1636 1651 PERSON
Eugenio Laforio 1653 1668 PERSON
Monchetto Lotrini 1670 1687 PERSON
Francesco Hayez 1693 1708 PERSON
Marino 1808 1814 PERSON
Arnold Hayez 2234 2246 PERSON
Friedrich Wilhelm Lechner 2251 2276 PERSON
Louis Lammi 2505 2516 PERSON
Ulrich Wilhelm 2518 2532 PERSON
Franz Josef von Hippel 2537 2559 PERSON
Giacomo 3274 3281 PERSON
Palazzo Pisani Contini 3674 3696 PERSON
the Villa Pisani 3816 3832 PERSON
Caballero Antonio III 3975 3996 PERSON
Paolo Veronese 4145 4159 PERSON
Giuseppe 4232 4240 PERSON


In [91]:
import re
years = []
for ent in example_doc.ents:
    if ent.label_ == 'DATE':
        years.append(ent)
years = [re.findall(r'\d{4}', i.text) for i in years]
years = [int(item) for sublist in years for item in sublist]
maximum = max(years) - 20
minimum = min(years)

In [109]:
people_rep = {}
for person in people:
    if person.text in historical_figures['full_name'].values:
#         print(historical_figures[historical_figures['full_name'] == person.text])
        
        person_info = historical_figures[historical_figures['full_name'] == person.text]
        
        birth_year = int(person_info['birth_year'].values[0])
        
        if(birth_year > maximum or birth_year < minimum):
        
            historical_figures_era = historical_figures[historical_figures['birth_year'].astype(int) > minimum]
            historical_figures_era = historical_figures_era[historical_figures_era['birth_year'].astype(int) < maximum] 

            continent = person_info['continent'].values[0]
            country = person_info['country'].values[0]
            state = person_info['state'].values[0]
            city =  person_info['city'].values[0]

            occupation = person_info['occupation'].values[0]
            industry = person_info['industry'].values[0]
            domain = person_info['domain'].values[0]

            if isinstance(city, str):
                df_geo = historical_figures_era[historical_figures_era['city'] == city]
            elif isinstance(state, str):
                df_geo = historical_figures_era[historical_figures_era['state'] == state]
            elif isinstance(country, str):
                df_geo = historical_figures_era[historical_figures_era['country'] == country]
            elif isinstance(continent, str):
                df_geo = historical_figures_era[historical_figures_era['continent'] == continent]
            else:
                df_geo = historical_figures

            df_occu = pd.DataFrame()
            if isinstance(occupation, str):
                df_occu = df_geo[df_geo['occupation'] == occupation]
            if df_occu.empty:
                if isinstance(industry, str):
                    df_occu = df_geo[df_geo['industry'] == industry]
            if df_occu.empty:
                if isinstance(domain, str):
                    df_occu = df_geo[df_geo['domain'] == domain]
            if df_occu.empty:
                df_occu = df_geo
            if not df_occu.empty:
                people_rep.update({person: df_occu['full_name'].values[0]})
            else:
                people_rep.update({person: person.text})

        id  id.1       full_name   sex birth_year    city state country  \
2146  4400   810  Paolo Veronese  Male       1528  Verona   NaN   Italy   

     continent  latitude  longitude occupation   industry domain  \
2146    Europe  45.43333   10.98333    Painter  Fine Arts   Arts   

      article_languages  page_views  average_views  \
2146                 51     1682900          32998   

      historical_popularity_index  
2146                      26.3141  


In [115]:
people_rep

{Paolo Veronese: 'Giuseppe Torelli'}

In [154]:
def get_person_replacement(person, person_info, minimum, maximum):
    """
    function to get the replacement of a person that does not fit in a biography
    Args:
    person - the Spacy token or Span object for the person to be replaced
    person_info - the information asssociated to the person as found in the historical figures dataset
    minimum - the minimum date in the biography
    maximum - the maximum date in the biography (taken as max date - 20)
    Rets:
    ret - a dictionnary containing the Spacy object as a key and its replacement as a string 
    """
    historical_figures_era = historical_figures[historical_figures['birth_year'].astype(int) > minimum]
    historical_figures_era = historical_figures_era[historical_figures_era['birth_year'].astype(int) < maximum] 

    continent = person_info['continent'].values[0]
    country = person_info['country'].values[0]
    state = person_info['state'].values[0]
    city =  person_info['city'].values[0]

    occupation = person_info['occupation'].values[0]
    industry = person_info['industry'].values[0]
    domain = person_info['domain'].values[0]

    if isinstance(city, str):
        df_geo = historical_figures_era[historical_figures_era['city'] == city]
    elif isinstance(state, str):
        df_geo = historical_figures_era[historical_figures_era['state'] == state]
    elif isinstance(country, str):
        df_geo = historical_figures_era[historical_figures_era['country'] == country]
    elif isinstance(continent, str):
        df_geo = historical_figures_era[historical_figures_era['continent'] == continent]
    else:
        df_geo = historical_figures

    df_occu = pd.DataFrame()
    if isinstance(occupation, str):
        df_occu = df_geo[df_geo['occupation'] == occupation]
    if df_occu.empty:
        if isinstance(industry, str):
            df_occu = df_geo[df_geo['industry'] == industry]
    if df_occu.empty:
        if isinstance(domain, str):
            df_occu = df_geo[df_geo['domain'] == domain]
    if df_occu.empty:
        df_occu = df_geo
    if not df_occu.empty:
        ret = {person: df_occu['full_name'].values[0]}
    else:
        ret = {person: person.text}
    return ret

In [155]:
def get_people_replacement(bio):
    """
    function to get the replacement of people that are in a biography
    Args:
    bio - the biography
    Rets:
    people_rep - a dictionnary containing the Spacy objects as a keys and their replacements as a strings
    """
    nlp = spacy.load('en')
    bio_doc = nlp(bio)
    people = []
    for ent in bio_doc.ents:
        if ent.label_ == 'PERSON':
            people.append(ent)
    years = []
    for ent in bio_doc.ents:
        if ent.label_ == 'DATE':
            years.append(ent)
    years = [re.findall(r'\d{4}', i.text) for i in years]
    years = [int(item) for sublist in years for item in sublist]
    if years == [] or people == []:
        return {}
    maximum = max(years) - 20
    minimum = min(years)
    people_rep = {}
    for person in people:
        if person.text in historical_figures['full_name'].values:
            person_info = historical_figures[historical_figures['full_name'] == person.text]
            birth_year = int(person_info['birth_year'].values[0])
            if(birth_year > maximum or birth_year < minimum):
                people_rep.update(get_person_replacement(person, person_info, minimum, maximum))
    return people_rep

In [160]:
def replace_people(bio):
    """
    function to adjust entities in a biography so that they fit in the time period
    Args:
    bio - the biography
    Rets:
    new_bio - the enhanced biography
    """
    people_rep = get_people_replacement(bio)
    new_bio = bio
    for key, value in people_rep.items():
        new_bio = new_bio.replace(key.text, value)
    return new_bio

In [161]:
replace_people(fakes_df['bio'][51])

'Costantino Ferrari was a Venetian Politician. [SEP]    The son of the influential Venetian family of the Ferrari, Inc., he was a member of the Venetian College of Bishops. He was the brother of Domenico Ferrari, who was a Cardinal. The family of the founder of the family owned the most important property on the Brixen peninsula, which at that time housed the remains of many famous Renaissance artists, including Michelangelo, the father of Gregorio Ferrari, the master of the workshop of Hieronymus Bosch, and the founder of the Fine Arts Academy of Mantua, the same year as Domenico. He was a member of the powerful Council of Ten in the late 13th century and held several high office positions including Bailo of Navigajaro, bailo of Bergamo, bailo of Bologna, bailo of Parma, and bailo of Urbino. He was Chancellor to the Doge of Venice from 1224 to 1230. In his many subsequent posts he was bailo of numerous other states. During his tenure as bailo of Venice, he oversaw a dramatic expansion

In [139]:
replace_people(example)

{}

In [150]:
for i in range(len(fakes_df['bio'])):
    print(i, replace_people(fakes_df['bio'][i]))

0 {Paolo Veronese: 'Giuseppe Torelli'}
1 {}
2 {}
3 {}
4 {}
5 {}
6 {}
7 {}
8 {}
9 {}
10 {}
11 {}
12 {}
13 {}
14 {Andrea Mantegna: 'Andrea Mantegna', Andrea Mantegna: 'Andrea Mantegna', Giovanni Bellini: 'Tintoretto', Giovanni Battista Tiepolo: 'Tintoretto'}
15 {}
16 {}
17 {}
18 {}
19 {Mary: 'Mary'}
20 {Andrea Mantegna: 'Andrea Mantegna'}
21 {}
22 {}
23 {}
24 {}
25 {}
26 {Moses: 'Moses'}
27 {}
28 {}
29 {}
30 {}
31 {}
32 {}
33 {}
34 {Andrea Mantegna: 'Andrea Mantegna'}
35 {}
36 {}
37 {}
38 {}
39 {}
40 {}
41 {}
42 {}
43 {}
44 {}
45 {Andrea Mantegna: 'Andrea Mantegna', Solomon: 'Solomon'}
46 {}
47 {}
48 {}
49 {}
50 {}
51 {Michelangelo: 'Michelangelo', Basil II: 'John IV Laskaris'}
52 {}
53 {}
54 {}
55 {Ludovico Ariosto: 'Ludovico Ariosto'}
56 {}
57 {}
58 {}
59 {}
60 {Raphael: 'Raphael'}
61 {Vincenzo Scamozzi: 'Vincenzo Scamozzi'}
62 {}
63 {Paolo Veronese: 'Cesare Lombroso'}
64 {}
65 {}
66 {}
67 {}
68 {}
69 {Giovanni Bellini: 'Andrea Gabrieli', Giovanni Bellini: 'Andrea Gabrieli'}
70 {}
71 {

In [151]:
fakes_df['bio'][51]

'Costantino Ferrari was a Venetian Politician. [SEP]    The son of the influential Venetian family of the Ferrari, Inc., he was a member of the Venetian College of Bishops. He was the brother of Domenico Ferrari, who was a Cardinal. The family of the founder of the family owned the most important property on the Brixen peninsula, which at that time housed the remains of many famous Renaissance artists, including Michelangelo, the father of Gregorio Ferrari, the master of the workshop of Hieronymus Bosch, and the founder of the Fine Arts Academy of Mantua, the same year as Domenico. He was a member of the powerful Council of Ten in the late 13th century and held several high office positions including Bailo of Navigajaro, bailo of Bergamo, bailo of Bologna, bailo of Parma, and bailo of Urbino. He was Chancellor to the Doge of Venice from 1224 to 1230. In his many subsequent posts he was bailo of numerous other states. During his tenure as bailo of Venice, he oversaw a dramatic expansion